In [1]:
cd ..

/Users/pierrerouanet/Projects/Arcadeep/tetris-over-9000


In [2]:
from dataset_loader import TetrisDataset

dataset = TetrisDataset('./traj-experts/dataset-10.npz', batch_size=32, 
                        randomize=False, shuffle=False)

In [3]:
from tetris import TetrisEnv
from stable_baselines.common.vec_env import DummyVecEnv

env = TetrisEnv()
env = DummyVecEnv([lambda: env])

In [4]:
from cnn import cnn

from stable_baselines import A2C
from stable_baselines.common.policies import FeedForwardPolicy

class CnnPolicy(FeedForwardPolicy):
    def __init__(self, *args, **kwargs):
        FeedForwardPolicy.__init__(self, *args, **kwargs, 
                                   cnn_extractor=cnn, feature_extraction='cnn')

model = A2C(CnnPolicy, env, verbose=1, tensorboard_log='./log')

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


In [5]:
import time

t0 = time.time()
model.pretrain(dataset, n_epochs=3)
t1 = time.time()

t1 - t0

Pretraining with Behavior Cloning...
==== Training progress 33.33% ====
Epoch 1
Training loss: 0.539359, Validation loss: 0.849644

==== Training progress 66.67% ====
Epoch 2
Training loss: 0.322712, Validation loss: 0.609532

==== Training progress 100.00% ====
Epoch 3
Training loss: 0.272631, Validation loss: 0.332581

Pretraining done.


166.58792090415955

In [6]:
import time

done = False
obs = env.reset()

while not done:
    action, states = model.predict(obs)
    obs, rew, done, info = env.step(action)
    env.render()
    time.sleep(1/30)